In [1]:
from init import *
import time

In [2]:
model = Model()

In [3]:
m = mcsdk.examples.BuildModel_Simple()
m.evaluations.append(EvaluationPoint(0,2))
m.json()

'{"TimeStart": 0, "TimeSteps": 10, "NumPaths": 10000, "updaters": [{"name": "IndependentGaussian"}, {"name": "BrownianMotion", "start": 0.1, "args": [0.2, 2]}], "evaluations": [{"state": 0, "time": 2, "histograms": []}], "RandomSeed": -1, "RunTimeoutSeconds": 1, "MemoryLimitKB": 64}'

In [4]:
r = requests.post(f'{server}/model',m.json())
r.json()

{'histograms': [],
 'mean': [0.5107610821723938],
 'names': ['BrownianMotion'],
 'npaths': [10000],
 'run_time_seconds': {'mean': 0.00029122785781510174,
  'n': 1024,
  'skewness': None,
  'stddev': 3.8849304928589845e-07},
 'skewness': [0.017882073298096657],
 'stddev': [2.8374462127685547],
 'time_points': [2.0],
 'time_steps': [10]}

In [5]:
m = mcsdk.examples.BuildModel_Simple()
m.NumPaths = 1000000
m.TimeSteps = 10000
m.RunTimeoutSeconds = 16.7
for t in [1,2,5,10]:
    m.evaluations.append(EvaluationPoint(0,t))
time_start = time.time()
r = requests.post(f'{server}/model',m.json())
time_end = time.time()
r.json()

{'error': ['../src/core/Model.cpp Model& Model::Run()',
  'Statistics model_version5(Cuda::StreamGuard&, KernelModelArgs&)',
  'Model execution timeout is reached.']}

In [6]:
print(f'{m.NumPaths*m.TimeSteps/(time_end-time_start)*1e-9:.1f} nanoseconds to make a single time step')

10.5 nanoseconds to make a single time step


# Two processes, Three evaluations

In [7]:
model = mcsdk.examples.TwoProcessesModel()
model.TimeSteps = 10000
for t in [1,2,5]:
    model.evaluations.append(EvaluationPoint(state=0,time=t))
r = requests.post(f'{server}/model',model.json())
r.json()

{'histograms': [],
 'mean': [0.3134157061576843,
  0.3093373477458954,
  0.5131708979606628,
  0.5087082982063293,
  1.0725574493408203,
  1.095420002937317],
 'names': ['BrownianMotion', 'BrownianMotion'],
 'npaths': [10000, 10000, 10000, 10000, 10000, 10000],
 'run_time_seconds': {'mean': 0.24365513026714325,
  'n': 1024,
  'skewness': None,
  'stddev': 0.00017287441005464643},
 'skewness': [0.02289663814008236,
  0.03331878036260605,
  0.0035845707170665264,
  0.008947157301008701,
  0.03021305799484253,
  0.028914714232087135],
 'stddev': [1.976637840270996,
  1.9868468046188354,
  2.8452250957489014,
  2.832554340362549,
  4.449069499969482,
  4.4814019203186035],
 'time_points': [1.0, 2.0, 5.0],
 'time_steps': [2000, 4000, 10000]}

In [8]:
er = EvaluationResults(r.json(),model)
er

2 states with 3 evaluations

In [9]:
er.df()

,name,title,state,time,step,npaths,mean,mean_error,stddev,skewness
0,BrownianMotion,,0,1.0,2000,10000,0.313416,0.019766,1.976638,0.022897
1,BrownianMotion,,1,1.0,2000,10000,0.309337,0.019868,1.986847,0.033319
2,BrownianMotion,,0,2.0,4000,10000,0.513171,0.028452,2.845225,0.003585
3,BrownianMotion,,1,2.0,4000,10000,0.508708,0.028326,2.832554,0.008947
4,BrownianMotion,,0,5.0,10000,10000,1.072557,0.044491,4.449069,0.030213
5,BrownianMotion,,1,5.0,10000,10000,1.095420,0.044814,4.481402,0.028915


In [10]:
logging.info('The End')

2022-10-17 15:59:08,213 [INFO] The End
